In [31]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install scikit-learn==0.24.1


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 15.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  error: subprocess-exited-with-error
  
  × Building wheel for scikit-learn (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for scikit-learn
Failed to build scikit-learn
ERROR: Could not build wheels for scikit-learn, which is required to install pyproject.toml-based projects


In [48]:
%%writefile predict_developers.py

import nltk
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import joblib
import sys
import json
import numpy as np

# Remove stop words from the bug report
def remove_stopwords(tokens, stop_words):
    return [word for word in tokens if word.lower() not in stop_words]

# Apply stemming on the bug tokens
def stem_tokens(tokens, stemmer):
    return [stemmer.stem(word) for word in tokens]

# Get the class using the svm_classifier
def predict_summary(summary, tfidf_vectorizer, svm_classifier):
    # Preprocess the input summary
    summary_tfidf = tfidf_vectorizer.transform(summary)

    # Predict the class
    predicted_class = svm_classifier.predict(summary_tfidf)

    return predicted_class[0]

# Define the prediction function to get top 5 classes
def predict_top_5_classes(summary, tfidf_vectorizer, svm_classifier, label_encoder):
    summary_tfidf = tfidf_vectorizer.transform([' '.join(summary)])
    probabilities = svm_classifier.predict_proba(summary_tfidf)[0]
    top_5_indices = np.argsort(probabilities)[-5:][::-1]
    return set(top_5_indices)

def Inference(bug_report, stop_words, stemmer, tfidf_vectorizer, svm_classifier, label_encoder):
    # Apply tokenization
    tokens = word_tokenize(bug_report)
    # Remove stop words
    remove_stopwords(tokens, stop_words)
    # Apply stemming
    stem_tokens(tokens, stemmer)
    # Predict the top 5 classes using the svm_classifier after applying TF-IDF
    top_5_classes = predict_top_5_classes(tokens, tfidf_vectorizer, svm_classifier, label_encoder)

    return top_5_classes

def main():
    # Prepare tokenizer, stemmer and stop words**
    nltk.download('punkt')
    nltk.download('stopwords')
    stop_words = set(stopwords.words('english'))
    stemmer = PorterStemmer()

    # Load the claddifier, TF-IDF vectorizer and label encoder
    svm_classifier = joblib.load('/content/drive/My Drive/checkpoints/svm_classifier_model_with_probability.joblib')
    tfidf_vectorizer = joblib.load('/content/drive/My Drive/checkpoints/tfidf_vectorizer.joblib')
    label_encoder = joblib.load('/content/drive/My Drive/checkpoints/label_encoder.joblib')

    # Prepare the data
    data = json.loads(sys.argv[1])
    # Extract bug description
    bug_description = data["bugDescription"]

    input_bug_top_5_classes = Inference(bug_description, stop_words, stemmer, tfidf_vectorizer, svm_classifier, label_encoder)

    # Create a dictionary to store the results
    developers_bugs_classes = {}

    developers_with_no_bugs = set()
    # Extract developers' data
    for developer in data["developersData"]:
        developer_id = developer["developerID"]
        old_bugs = developer["oldBugsDescription"]
        # Check if the bug list is empty
        if not old_bugs:
            developers_with_no_bugs.add(developer_id)
            continue

        common_classes_count = 0
        for bug in old_bugs:
            predicted_top_5_classes = Inference(bug, stop_words, stemmer, tfidf_vectorizer, svm_classifier, label_encoder)
            common_classes = input_bug_top_5_classes.intersection(predicted_top_5_classes)
            common_classes_count += len(common_classes)
        developers_bugs_classes[developer_id] = common_classes_count

    # Sort developers by common class counts in descending order
    sorted_developers = sorted(developers_bugs_classes.items(), key=lambda item: item[1], reverse=True)

    # Get the top developers (at most 5)
    num_top_developers = min(5, len(sorted_developers))

    # Get the top 5 developers
    recommended_developers = [developer_id for developer_id, _ in sorted_developers[:num_top_developers]]

    # If there are developers that didn't solve bugs before ==> add them at the end of the list so they try to solve bugs
    if developers_with_no_bugs:
      if(len(recommended_developers)==5):
        # replace the last recommendation in the recommended develoeprs with a developer from the team that didn't solve a bug before
        recommended_developers[4] = developers_with_no_bugs.pop()
      else:
        # if the recommended_developers list is less than 5 recommendations ==> add developers with no bugs till their set ends or the recommendations list reaches 5 developers
        while(len(recommended_developers) < 5 and developers_with_no_bugs):
          recommended_developers.append(developers_with_no_bugs.pop())

    # Print the results
    print("Recommended Developers:", recommended_developers)

if __name__ == "__main__":
    main()

Overwriting predict_developers.py


In [47]:
!python predict_developers.py '{"bugDescription": "Maximize on second larger monitor not working ", "developersData": [{"developerID": "667a76f471631147e0b6e0d", "jobTitle": "Backend Developer", "oldBugsDescription": []},{"developerID": "667a76f471631147e0b6e0ddd", "jobTitle": "Backend Developer", "oldBugsDescription": []},{"developerID": "667a76f471631147e0b6e0dbbd", "jobTitle": "Backend Developer", "oldBugsDescription": []},{"developerID": "667a76f471631147e0b6e0deedd", "jobTitle": "Backend Developer", "oldBugsDescription": []},{"developerID": "667a760a6da0c47fe0a327cd", "jobTitle": "Backend Developer", "oldBugsDescription": ["Maximize on second larger monitor not working", "the font size is very small"]}, {"developerID": "667a76f471631147e0b6e0d6", "jobTitle": "Backend Developer", "oldBugsDescription": ["Manual guide installation is not clear"]}]}'

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator SVC from version 0.24.2 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator TfidfTransformer from version 0.24.2 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/usr/local/lib/python